In [1]:
import pandas as pd 

## Read datasets

In [2]:
df = pd.read_csv("../data/ads_clean.csv")
dfs = pd.read_csv("../data/stations.csv")
data = df.copy()

In [3]:
dfs.head()

id  ad_id                       name      primaryType  \
0   1      6           Tenay Hauteville  transit_station   
1   2      7  Gare de Ambérieu-en-Bugey  transit_station   
2   3     10            Bourg-en-Bresse  transit_station   
3   4     10            BOURG Gare SNCF         bus_stop   
4   5     10   Bourg en Bresse, Charité         bus_stop   

                                               types   latitude  longitude  \
0  transit_station;train_station;bus_stop;point_o...  45.922258   5.501550   
1  transit_station;train_station;bus_stop;point_o...  45.953949   5.342462   
2  transit_station;bus_stop;train_station;point_o...  46.200099   5.214971   
3  bus_stop;transit_station;point_of_interest;est...  46.198810   5.215560   
4  bus_stop;transit_station;point_of_interest;est...  46.207630   5.219742   

   distance  
0  0.717523  
1  1.872351  
2  0.503876  
3  0.385902  
4  1.160297

In [4]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135623 entries, 0 to 135622
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           135623 non-null  int64  
 1   ad_id        135623 non-null  int64  
 2   name         135623 non-null  object 
 3   primaryType  135623 non-null  object 
 4   types        135623 non-null  object 
 5   latitude     135623 non-null  float64
 6   longitude    135623 non-null  float64
 7   distance     135623 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 8.3+ MB


In [5]:
primary_types = dfs["primaryType"].unique()
dfs["primaryType"].value_counts()

primaryType
bus_stop           101479
subway_station      11509
transit_station     10587
train_station        7276
bus_station          4772
Name: count, dtype: int64

In [6]:
types = set()
for i, row in dfs.iterrows():
    types_ = row["types"].split(";")
    for t in types_:
        types.add(t)
types

{'bus_station',
 'bus_stop',
 'establishment',
 'ferry_terminal',
 'light_rail_station',
 'point_of_interest',
 'school',
 'subway_station',
 'train_station',
 'transit_station'}

## Create one hot encoding for types

In [7]:
for t in primary_types:
    dfs[t] = 0
    for i, row in dfs.iterrows():
        if t in row["types"]:
            dfs.at[i, t] = 1

In [8]:
dfs.head()

id  ad_id                       name      primaryType  \
0   1      6           Tenay Hauteville  transit_station   
1   2      7  Gare de Ambérieu-en-Bugey  transit_station   
2   3     10            Bourg-en-Bresse  transit_station   
3   4     10            BOURG Gare SNCF         bus_stop   
4   5     10   Bourg en Bresse, Charité         bus_stop   

                                               types   latitude  longitude  \
0  transit_station;train_station;bus_stop;point_o...  45.922258   5.501550   
1  transit_station;train_station;bus_stop;point_o...  45.953949   5.342462   
2  transit_station;bus_stop;train_station;point_o...  46.200099   5.214971   
3  bus_stop;transit_station;point_of_interest;est...  46.198810   5.215560   
4  bus_stop;transit_station;point_of_interest;est...  46.207630   5.219742   

   distance  transit_station  bus_stop  train_station  bus_station  \
0  0.717523                1         1              1            0   
1  1.872351                1         1              1            0   
2  0.503876                1         1              1            0   
3  0.385902                1         1              0            0   
4  1.160297                1         1              0            0   

   subway_station  
0               0  
1               0  
2               0  
3               0  
4               0

## Basic Score

- for each ad: score of giving type of station = number of occurrences

In [9]:
for t in primary_types:
    data[t] = 0
    for i, row in data.iterrows():
        score = 0
        dfs_ = dfs[dfs["ad_id"] == row["id"]]
        if dfs_.shape[0] == 0:
            continue
        for j, row_ in dfs_.iterrows():
            score += row_[t]
        data.at[i, t] = score   

In [10]:
data.head()

id department_name  department_number          city_name  ad_type  \
0   4             ain                  1  Amberieu en Bugey        2   
1   5             ain                  1  Amberieu en Bugey        2   
2   6             ain                  1  Amberieu en Bugey        2   
3   7             ain                  1  Amberieu en Bugey        3   
4  10             ain                  1             Viriat        3   

   latitude  longitude   price  area  energy_DPE  ...  Proximité commerces  \
0   45.8130    5.18679   620.0  40.0           5  ...                    0   
1   45.9214    5.23109   795.0  70.0           5  ...                    0   
2   45.9196    5.51000   420.0  33.0           4  ...                    1   
3   45.9682    5.35535   990.0  99.0           4  ...                    1   
4   46.1972    5.22000  1000.0  80.0           5  ...                    1   

   Jardin  Plusieurs salles de bains  Cuisine indépendante  Ascenseur  \
0       0                          0                     0          0   
1       0                          0                     0          0   
2       0                          0                     0          0   
3       0                          1                     0          0   
4       0                          0                     1          0   

   transit_station  bus_stop  train_station  bus_station  subway_station  
0                0         0              0            0               0  
1                0         0              0            0               0  
2                1         1              1            0               0  
3                1         1              1            0               0  
4                8         7              2            0               0  

[5 rows x 40 columns]

In [11]:
data.to_csv("../data/ads_stations_basic.csv", index=False)

## Weighted Score

now the scores are weighted with distances:

here are some possible score functions:

In [12]:
import math
# Given distance in kilometers
distances = [0.1, 0.5, 1, 1.5, 2]  # example distance

for distance in distances:
    print("=====================")
    # Inverse Distance Weighting
    k = 1
    inverse_weight = 1 / (distance ** k)

    # Gaussian Distance Weighting
    sigma = 0.5
    gaussian_weight = math.exp(-(distance ** 2) / (2 * sigma ** 2))

    # Logarithmic Distance Weighting
    base = 2
    logarithmic_weight = math.log(1 + (base - distance))

    # Linear Distance Weighting
    max_distance = 2
    linear_weight = max(0, max_distance - distance)

    # Display the scores for each method
    print(f"Scores for a Distance of {distance} km:")
    print("Inverse Distance Weighting:", inverse_weight)
    print("Gaussian Distance Weighting:", gaussian_weight)
    print("Logarithmic Distance Weighting:", logarithmic_weight)
    print("Linear Distance Weighting:", linear_weight)

Scores for a Distance of 0.1 km:
Inverse Distance Weighting: 10.0
Gaussian Distance Weighting: 0.9801986733067553
Logarithmic Distance Weighting: 1.0647107369924282
Linear Distance Weighting: 1.9
Scores for a Distance of 0.5 km:
Inverse Distance Weighting: 2.0
Gaussian Distance Weighting: 0.6065306597126334
Logarithmic Distance Weighting: 0.9162907318741551
Linear Distance Weighting: 1.5
Scores for a Distance of 1 km:
Inverse Distance Weighting: 1.0
Gaussian Distance Weighting: 0.1353352832366127
Logarithmic Distance Weighting: 0.6931471805599453
Linear Distance Weighting: 1
Scores for a Distance of 1.5 km:
Inverse Distance Weighting: 0.6666666666666666
Gaussian Distance Weighting: 0.011108996538242306
Logarithmic Distance Weighting: 0.4054651081081644
Linear Distance Weighting: 0.5
Scores for a Distance of 2 km:
Inverse Distance Weighting: 0.5
Gaussian Distance Weighting: 0.00033546262790251185
Logarithmic Distance Weighting: 0.0
Linear Distance Weighting: 0


In [29]:
from utils import inverse_distance_weighting, gaussian_distance_weighting, logarithmic_distance_weighting, linear_distance_weighting

In [30]:
data = df.copy()
    
for score in ["inverse_score", "gaussian_score", "logarithmic_score", "linear_score"]:
    for t in primary_types:
        data[score + "_" + t] = 0.0
        
# Parameters for weighting functions
k = 1  # for inverse distance weighting
sigma = 0.5  # for Gaussian distance weighting
base = 2  # for logarithmic distance weighting
max_distance = 2  # for linear distance weighting


In [31]:
for t in primary_types:
    
    for i, row in data.iterrows():
        
        score = 0
        dfs_ = dfs[dfs["ad_id"] == row["id"]]
        
        if dfs_.shape[0] == 0:
            continue
        
        distances = []
        for j, row_ in dfs_.iterrows():
                        
            if row_[t] == 0:
                continue
            distance = row_["distance"]           
            distances.append(distance)
            
        
        # Calculate the scores for each method
        data.at[i, "inverse_score" + "_" + t] = sum(inverse_distance_weighting(distances, k))
        data.at[i, "gaussian_score" + "_" + t] = sum(gaussian_distance_weighting(distances, sigma))
        data.at[i, "logarithmic_score" + "_" + t] = sum(logarithmic_distance_weighting(distances, base))
        data.at[i, "linear_score" + "_" + t] = sum(linear_distance_weighting(distances, max_distance))


In [32]:
data.describe()

id  department_number      ad_type     latitude    longitude  \
count   7524.000000        7524.000000  7524.000000  7524.000000  7524.000000   
mean    5117.885300          49.855529     1.528708    46.909318     3.104070   
std     2913.414568          28.289924     0.802992     2.319977     2.481272   
min        4.000000           1.000000     0.000000    41.675600    -4.760020   
25%     2662.750000          29.000000     1.000000    44.927225     2.039897   
50%     5197.500000          54.000000     2.000000    47.372750     2.881630   
75%     7643.250000          77.000000     2.000000    48.846050     4.887650   
max    10028.000000          94.000000     3.000000    51.078300     9.450330   

             price         area   energy_DPE   energy_GES    furnished  ...  \
count  7524.000000  7524.000000  7524.000000  7524.000000  7524.000000  ...   
mean    763.949495    42.607257     3.711457     3.375066     0.620946  ...   
std     419.936690    28.819483     1.037933     1.098593     0.485184  ...   
min     101.000000     9.000000     1.000000     1.000000     0.000000  ...   
25%     490.000000    21.000000     3.000000     3.000000     0.000000  ...   
50%     650.000000    35.000000     4.000000     4.000000     1.000000  ...   
75%     900.000000    58.000000     4.000000     4.000000     1.000000  ...   
max    4700.000000   392.000000     7.000000     7.000000     1.000000  ...   

       logarithmic_score_transit_station  logarithmic_score_bus_stop  \
count                        7524.000000                 7524.000000   
mean                           10.235987                    8.485231   
std                             3.848825                    4.404659   
min                             0.000000                   -0.001448   
25%                             8.530860                    5.557220   
50%                            10.545467                    9.117799   
75%                            12.775634                   11.712881   
max                            19.413627                   19.413627   

       logarithmic_score_train_station  logarithmic_score_bus_station  \
count                      7524.000000                    7524.000000   
mean                          0.860740                       0.377867   
std                           0.797635                       1.105978   
min                          -0.000065                      -0.001810   
25%                           0.115390                       0.000000   
50%                           0.768403                       0.000000   
75%                           1.255170                       0.344099   
max                           4.845521                      14.288633   

       logarithmic_score_subway_station  linear_score_transit_station  \
count                       7524.000000                   7524.000000   
mean                           0.963116                     15.090064   
std                            2.302963                      6.212566   
min                            0.000000                      0.000000   
25%                            0.000000                     11.813764   
50%                            0.000000                     15.160691   
75%                            0.000000                     19.256933   
max                           11.302464                     32.907194   

       linear_score_bus_stop  linear_score_train_station  \
count            7524.000000                 7524.000000   
mean               12.547042                    1.263804   
std                 6.948905                    1.204436   
min                 0.000000                    0.000000   
25%                 7.588764                    0.121159   
50%                12.975125                    1.098774   
75%                17.542233                    1.813694   
max                32.878488                    8.144950   

       linear_score_bus_station  linear_score_subway_station  
count 

In [33]:
data.to_csv("../data/ads_stations_weighted.csv", index=False)